In [14]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyodbc
from unidecode import unidecode
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import year, month
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from sqlalchemy import create_engine

# Extração dos dados 

Os dados foram extraídos pela ANTAQ para os anos de 2021, 2022 e 2023 e compilados utilizando o script abaixo

Extração automatizada via BeautifulSoup

In [ ]:
BASE_URL = "https://web3.antaq.gov.br/ea/sense/download.html#pt"
DOWNLOAD_URL = "https://web3.antaq.gov.br/ea/sense/"

anos = ["2021", "2022", "2023"] #Modificar caso necessário outros períodos
arquivos_desejados = ["Atracação", "Carga", "Carga Conteinerizada"]

os.makedirs("dados_brutos", exist_ok=True)

def baixar_arquivos():
    response = requests.get(BASE_URL)
    soup = BeautifulSoup(response.text, "html.parser")

    links = soup.find_all("a", string="Clique aqui.")

    for link in links:
        href = link.get("href")
        if href and href.endswith(".zip"):
            nome_arquivo = link.find_previous("td").text.strip()
            ano = href.split("/")[-2]

            if nome_arquivo in arquivos_desejados and ano in anos:
                arquivo_url = DOWNLOAD_URL + href
                salvar_caminho = os.path.join("dados_brutos", f"{nome_arquivo}_{ano}.zip")

                print(f"Baixando {arquivo_url}...")
                with open(salvar_caminho, "wb") as f:
                    f.write(requests.get(arquivo_url).content)
                
                print(f"Salvo: {salvar_caminho}")

if __name__ == "__main__":
    baixar_arquivos()

OBS. No advento de algum problema de conexão e download automático dos arquivos, é possível pular a etapa anterior e partir do script abaixo. Contanto que os dados estejam salvos na seguinte estrutura:

./dados_brutos/2021/2021Atracacao.txt
./dados_brutos/2021/2021Carga.txt
./dados_brutos/2021/2021Carga_Conteinerizada.txt
./dados_brutos/2022/2022Atracacao.txt
./dados_brutos/2022/2022Carga.txt
./dados_brutos/2022/2022Carga_Conteinerizada.txt
./dados_brutos/2023/2023Atracacao.txt
./dados_brutos/2023/2023Carga.txt
./dados_brutos/2023/2023Carga_Conteinerizada.txt

...

União dos arquivos via PySpark

In [4]:
spark = SparkSession.builder.appName("ExtracaoDados").getOrCreate()

anos = ["2021", "2022", "2023"] #Modificar caso necessário outros períodos

arquivos = ["Atracacao", "Carga", "Carga_Conteinerizada"]

input_dir = "dados_brutos"

atracacao = None
carga = None
carga_cont = None

for arquivo in arquivos:
    input_files = [f"{input_dir}/{ano}/{ano}{arquivo}.txt" for ano in anos if os.path.exists(f"{input_dir}/{ano}/{ano}{arquivo}.txt")]

    if not input_files:
        print("{arquivo} não encontrado. Indo para o arquivo seguinte...")
        continue
    
    df = spark.read.option("header", True).option("delimiter", ";").csv(input_files)

    if arquivo == "Atracacao":
        atracacao = df
    elif arquivo == "Carga":
        carga = df
    elif arquivo == "Carga_Conteinerizada":
        carga_cont = df  



# Tratamento de dados

## Tabela 01 (atracacao_fato)

Filtrar apenas pelas colunas de interesse (tabela_atracacao)

In [5]:
colunas_atracacao = [
    'IDAtracacao', 'Tipo de Navegação da Atracação', 'CDTUP', 'Nacionalidade do Armador', 
    'IDBerco', 'FlagMCOperacaoAtracacao', 'Berço', 'Terminal', 'Porto Atracação', 
    'Município', 'Apelido Instalação Portuária', 'UF', 'Complexo Portuário', 
    'SGUF', 'Tipo da Autoridade Portuária', 'Região Geográfica', 'Nº da Capitania', 
    'Nº do IMO', 'Data Atracação', 'Data Chegada', 'Data Desatracação', 'Data Início Operação', 
    'Data Término Operação', 'Tipo de Operação'
]

atracacao_fato = atracacao.select(*colunas_atracacao)



Remover valores nulos/ausentes

In [6]:
atracacao_fato = atracacao_fato.dropna()


Converter colunas para o tipo data

In [7]:
datas = ['Data Atracação', 'Data Chegada', 'Data Desatracação', 'Data Início Operação', 'Data Término Operação']

for coluna in datas:
    atracacao_fato = atracacao_fato.withColumn(coluna, to_timestamp(atracacao_fato[coluna], 'dd/MM/yyyy HH:mm:ss'))


Gerar as colunas 'Ano da data de início da operação' e 'Mês da data de início da operação'

In [8]:
atracacao_fato = atracacao_fato.withColumn('Ano da data de início da operação', year(atracacao_fato['Data Atracação']))

atracacao_fato = atracacao_fato.withColumn('Mês da data de início da operação', month(atracacao_fato['Data Atracação']))


# ----------------------------------------------------------------

## Tabela 02 (carga_fato)

Filtrar apenas pelas colunas de interesse (tabelas carga e atracacao_fato)

In [9]:
colunas_carga = [
    'IDCarga', 'FlagTransporteViaInterioir', 'IDAtracacao', 'Percurso Transporte em vias Interiores', 
    'Origem', 'Percurso Transporte Interiores', 'Destino', 'STNaturezaCarga', 
    'CDMercadoria', 'STSH2', 'Tipo Operação da Carga', 'STSH4', 'Carga Geral Acondicionamento', 
    'Natureza da Carga', 'ConteinerEstado', 'Sentido', 'Tipo Navegação', 'TEU', 'FlagAutorizacao', 
    'QTCarga'
]

colunas_atracacao_fato = [
    'IDAtracacao', 'Porto Atracação', 'SGUF', 'Ano da data de início da operação', 'Mês da data de início da operação'
]

carga_intermed = carga.select(*colunas_carga)
atracacao_fato_intermed = atracacao_fato.select(*colunas_atracacao_fato)

carga_fato = carga_intermed.join(atracacao_fato_intermed, on="IDAtracacao", how="left")



Calcular Peso líquido da carga:   
carga_merged['VLPesoLiquido'] = carga_merged['VLPesoCargaBruta'] - carga_merged['VLPesoCargaConteinerizada']

In [10]:
carga = carga.dropna()
carga_cont = carga_cont.dropna()

carga = carga.withColumn("VLPesoCargaBruta", F.regexp_replace("VLPesoCargaBruta", ",", ".").cast("float"))
carga_cont = carga_cont.withColumn("VLPesoCargaConteinerizada", F.regexp_replace("VLPesoCargaConteinerizada", ",", ".").cast("float"))

carga_merged = carga.join(carga_cont, on="IDCarga", how="left")
carga_merged = carga_merged.withColumn("VLPesoLiquido", F.col("VLPesoCargaBruta") - F.col("VLPesoCargaConteinerizada"))

carga_fato = carga_fato.join(carga_merged.select("IDCarga", "VLPesoLiquido"), on="IDCarga", how="left")


Remoção de valores nulos/ausentes

In [11]:
carga_fato = carga_fato.dropna()

# ----------------------------------------------------------------

Remover acentuação, cedilha, espaços e outros caracteres nos nomes das variáveis para correto carregamento no banco de dados

In [17]:
for coluna in atracacao_fato.columns:
    novo_nome = unidecode(coluna)
    atracacao_fato = atracacao_fato.withColumnRenamed(coluna, novo_nome)

for coluna in carga_fato.columns:
    novo_nome = unidecode(coluna)
    carga_fato = carga_fato.withColumnRenamed(coluna, novo_nome)

In [18]:
atracacao_fato = atracacao_fato.select(
    col("IDAtracacao"),
    col("Tipo de Navegacao da Atracacao").alias("Tipo_Navegacao_Atracacao"),
    col("CDTUP"),
    col("Nacionalidade do Armador").alias("Nacionalidade_Armador"),
    col("IDBerco"),
    col("FlagMCOperacaoAtracacao"),
    col("Berco"),
    col("Terminal"),
    col("Porto Atracacao").alias("Porto_Atracacao"),
    col("Municipio"),
    col("Apelido Instalacao Portuaria").alias("Apelido_Instalacao_Portuaria"),
    col("UF"),
    col("Complexo Portuario").alias("Complexo_Portuario"),
    col("SGUF"),
    col("Tipo da Autoridade Portuaria").alias("Tipo_Autoridade_Portuaria"),
    col("Regiao Geografica").alias("Regiao_Geografica"),
    col("No da Capitania").alias("No_Capitania"),
    col("No do IMO").alias("No_IMO"),
    col("Data Atracacao").alias("Data_Atracacao"),
    col("Data Chegada").alias("Data_Chegada"),
    col("Data Desatracacao").alias("Data_Desatracacao"),
    col("Data Inicio Operacao").alias("Data_Inicio_Operacao"),
    col("Data Termino Operacao").alias("Data_Termino_Operacao"),
    col("Tipo de Operacao").alias("Tipo_Operacao"),
    col("Ano da data de inicio da operacao").alias("Ano_Inicio_Operacao"),
    col("Mes da data de inicio da operacao").alias("Mes_Inicio_Operacao")
)

carga_fato = carga_fato.select(
    col("IDCarga"),
    col("IDAtracacao"),
    col("Origem"),
    col("Destino"),
    col("CDMercadoria"),
    col("Tipo Operacao da Carga").alias("Tipo_Operacao_Carga"),
    col("Carga Geral Acondicionamento").alias("Carga_Geral_Acondicionamento"),
    col("ConteinerEstado"),
    col("Tipo Navegacao").alias("Tipo_Navegacao"),
    col("FlagAutorizacao"),
    col("Percurso Transporte em vias Interiores").alias("Percurso_Transporte_Vias_Interiores"),
    col("Percurso Transporte Interiores").alias("Percurso_Transporte_Interiores"),
    col("STNaturezaCarga"),
    col("STSH2"),
    col("STSH4"),
    col("Natureza da Carga").alias("Natureza_Carga"),
    col("Sentido"),
    col("TEU"),
    col("QTCarga"),
    col("VLPesoLiquido").alias("VLPesoCargaBruta")
)

Visualização das tabelas finalizadas

In [19]:
atracacao_fato.show(3)
carga_fato.show(3)

+-----------+------------------------+-------+---------------------+-----------+-----------------------+-----------+--------------------+--------------------+---------+----------------------------+--------+--------------------+----+-------------------------+-----------------+------------+-------+-------------------+-------------------+-------------------+--------------------+---------------------+--------------------+-------------------+-------------------+
|IDAtracacao|Tipo_Navegacao_Atracacao|  CDTUP|Nacionalidade_Armador|    IDBerco|FlagMCOperacaoAtracacao|      Berco|            Terminal|     Porto_Atracacao|Municipio|Apelido_Instalacao_Portuaria|      UF|  Complexo_Portuario|SGUF|Tipo_Autoridade_Portuaria|Regiao_Geografica|No_Capitania| No_IMO|     Data_Atracacao|       Data_Chegada|  Data_Desatracacao|Data_Inicio_Operacao|Data_Termino_Operacao|       Tipo_Operacao|Ano_Inicio_Operacao|Mes_Inicio_Operacao|
+-----------+------------------------+-------+---------------------+--------

# ----------------------------------------------------------------

# Disponibilização em banco de dados (SQL Server)

Configuração da conexão

In [ ]:
server = "XXXXXXXXXXXX"
database = "fiec_antaq"
username = "XXXXXXXXXXXXXXX"
password = "XXXXXXXXXXXXXXX"

Importação dos dados

In [ ]:
connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=SQL+Server"
engine = create_engine(connection_string, fast_executemany=True)

# Conversão para Pandas:
atracacao_fato_pd = atracacao_fato.toPandas()
carga_fato_pd = carga_fato.toPandas()

def importar_dados_para_sql(df, tabela):
    try:
        df.to_sql(tabela, engine, if_exists="append", index=False, chunksize=1000)
    except Exception as e:
        print("Erro ao importar os dados para a tabela {tabela}: {e}")

importar_dados_para_sql(atracacao_fato_pd, "atracacao_fato")
importar_dados_para_sql(carga_fato_pd, "carga_fato")

engine.dispose()